In [3]:
import numpy as np
from numba import njit, jit, vectorize, float64, prange
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
from os.path import exists
from scipy import stats

mpl.rcParams["figure.dpi"] = 180
mpl.rcParams["legend.fontsize"] = 11
mpl.rcParams["font.size"] = 11
plt.rc("text", usetex=False)
plt.rc("text.latex", preamble=r"""
\usepackage{palatino}
\usepackage{newpxmath}""")
plt.rc("font", family="serif")

In [4]:
def confidence_intervals(sample_variances, p):
    """
    Returns the 100(1-p)% confidence intervals
    sample_variances    ----------
    sample_variances: array
        array of sample variances
    
    p: float between 0 and 1 exclusive
        p-value for confidence interval
        
    Returns
    -------
    intervals: array of floats
        |x - μ| < intervals, where X is the sample mean
        
    Notes
    -----
    Calculated as
        `stats.t.ppf(q = 1 - p/2, df=n-1) * np.sqrt(sample_variances)/np.sqrt(n)`
    where n = `len(sample_variances)`
    """
    n = len(sample_variances)
    intervals = stats.t.ppf(q = 1 - p/2, df=n-1) * np.sqrt(sample_variances)/np.sqrt(n)
    return intervals

In [5]:
@njit
def random_uniform(low = 0.0, high = 1.0, size = 1, seed = 0):
    if (seed):
        np.random.seed(seed)
        
    randos = np.random.random(size = size)
    scale = np.abs(high - low)
    randos *= scale
    randos += low
    return randos

@njit
def random_uniform_orthogonal(nax, l_bounds, u_bounds, seed = 0):
    """
    TODO: license
    TODO: docstring
    TODO: seeding?
    """
    if (seed):
        np.random.seed(seed)
        
    N = np.power(nax, 2)
    x_scale = (u_bounds[0] - l_bounds[0])/N
    y_scale = (u_bounds[1] - l_bounds[1])/N
    x0 = l_bounds[0]
    y0 = l_bounds[1]
    xlist = np.arange(N).reshape((nax,nax))
    ylist = np.arange(N).reshape((nax,nax))
    x_randos = np.empty(N, dtype = np.float64)
    y_randos = np.empty(N, dtype = np.float64)
    for i in range(nax):
        xlist[i] = np.random.permutation(xlist[i])
        ylist[i] = np.random.permutation(ylist[i])
    for i in range(nax):
        for j in range(nax):
            x_randos[i*nax + j] = x0 + x_scale * (xlist[i][j] + (np.random.random()))
            y_randos[i*nax + j] = y0 + y_scale * (ylist[j][i] + (np.random.random()))
    return x_randos, y_randos

In [6]:
@jit(nopython = True)
def default_escape_cond(z):
    return np.abs(z) >= 2

@jit(nopython = True)
def main_cardioid(arg):
    point = 0.5 * np.exp(1j*arg) - 0.25 * np.exp(2*1j*arg)
    return np.abs(point)

#@jit(nopython = True)
#def cardioid_escape_cond(z):
    #if np.abs(z) <= main_cardioid(z.)

def mandelbrot_batch_factory(escape_condition):
    @jit(nopython = True, parallel = True, nogil = True)
    def _mandelbrot_mc(max_iter, samples):
        iter_counts = np.empty(len(samples), dtype = np.int64)
        for i in prange(len(samples)):
            c = samples[i]
            zn = c
            for count in range(max_iter):
                zn = zn * zn + c
                if (escape_condition(zn)):
                    iter_counts[i] = count
                    break
                elif count == max_iter - 1:
                    iter_counts[i] = count
        return iter_counts
    return _mandelbrot_mc

mandelbrot_mc_default = mandelbrot_batch_factory(default_escape_cond)

@njit
def create_batch_uniform(N, re_low, re_high, im_low, im_high, seed = 0):
    if (seed):
        np.random.seed(seed)
    samples_re = random_uniform(low = re_low, high = re_high, size = N)
    samples_im = random_uniform(low = im_low, high = im_high, size = N)
    samples = samples_re + 1j*samples_im
    return samples

@njit
def create_batch_orthogonal(N, re_low, re_high, im_low, im_high, seed = 0):
    if (seed):
        np.random.seed(seed)
    nax = np.int64(np.ceil(np.sqrt(N)))
    l_bounds = (re_low, im_low)
    u_bounds = (re_high, im_high)
    samples_re, samples_im = random_uniform_orthogonal(nax, l_bounds, u_bounds)
    samples = samples_re + 1j*samples_im
    samples = samples[:N]
    return samples

def create_batch_lhc(N, re_low, re_high, im_low, im_high, seed = 0):
    if (seed):
        sampler = stats.qmc.LatinHypercube(d = 2, seed = seed)
    else:
        sampler = stats.qmc.LatinHypercube(d = 2)
    re_scale = re_high - re_low
    im_scale = im_high - im_low
    samples = sampler.random(N)
    samples[:,0] *= re_scale
    samples[:,1] *= im_scale
    samples[:,0] += re_low
    samples[:,1] += im_low
    samples = samples[:,0] + 1j*samples[:,1]
    return samples

@njit
def genrand_uniform(low = 0.0, high = 1.0, size = 1, sampler = random_uniform):
    return sampler(low, high, size)

In [7]:
@jit(nopython = True)
def set_count_from_iter_counts(max_iter, iter_counts):
    set_count = 0
    for count in iter_counts:
        if count == max_iter - 1:
            set_count += 1
    return set_count

@jit(nopython = True)
def area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, max_iter):
    n = len(iter_counts)
    set_count = set_count_from_iter_counts(max_iter, iter_counts)
    domain_area = (re_high - re_low) * (im_high - im_low)
    return domain_area * set_count / n

In [9]:
@jit(nopython = True, parallel = True, nogil = True)
def mandelbrot_mc(max_iter, samples):
    """
    Counts the number of complex numbers in `samples` that do
    not meet the escape condition in `max_iter` recursions of
    the Mandelbrot polynomial, z(n+1) = z(n)^2 + c
    
    The escape condition used is |z(n)| >= 2
    
    Parameters
    ----------
    max_iter: positive integer
        Value of n up to which z(n) is evaluated
    
    samples: array of complex numbers
        values of z(0)
        
    Returns
    -------
    set_count: positive integer
        Number of complex numbers in `samples` that remain
        that do not satisfy the escape condition
    """
    set_count = 0
    for i in prange(len(samples)):
        c = samples[i]
        zn = c
        for j in range(1, max_iter):
            zn = zn*zn + c
            if (np.abs(zn) > 2):
                # numba should recognize this as a critical section
                break
            elif j == max_iter - 1:
                set_count += 1
    return set_count

@jit(nopython = True, nogil = True, parallel = True)
def mandelbrot_mc_area(re_low, re_high, im_low, im_high, max_iter, samples):
    """
    Calculates estimator of the area of the Mandelbrot set
    
    Parameters
    ----------
    re_low, re_high, im_low, im_high: float or float-like
        corners of bounding rectangle in the complex plane
    
    max_iter: positive integer
        value of n up to which z(n) is evaluated
    
    samples: array of complex numbers
        list of c values, assumed to be drawn from a uniform distribution
        in the bounding region
        
    Returns
    -------
    area: default numpy float (float32 or float64)
        estimate of the area of the mandelbrot set
    """
    count = mandelbrot_mc(max_iter, samples)
    return mandelbrot_area(count, len(samples), re_low, re_high, im_low, im_high)

@jit(nopython = True)
def mandelbrot_area(count, N, re_low, re_high, im_low, im_high):
    """
    TODO: Add docstring
    """
    rect_area = (re_high - re_low) * (im_high - im_low)
    return rect_area * count / N

@jit(nopython = True)
def mandelbrot_mc_runs(max_iter, samples, runs_count):
    counts = np.zeros(runs_count)
    for _ in range(runs_count):
        counts[i] = mandelbrot_mc(max_iter, samples)
    return counts

@jit(nopython = True)
def sample_mean_variance(counts):
    """Returns sample mean and sample variance of input array
    Parameters
    ----------
    counts: array of numbers
    
    Returns: 2-tuple (sample mean, sample variance)
    """
    n = len(counts)
    assert n > 1
    sample_mean = np.mean(counts)
    sample_variance = np.sum(np.power(counts - sample_mean, 2))/(n - 1)
    return (sample_mean, sample_variance)

In [10]:
N = 10**6
i = 1000
re_low, re_high = -2., 1.
im_low, im_high = 0, 1.5
area = (re_high - re_low) * (im_high - im_low)
samples = create_batch_uniform(N, re_low, re_high, im_low, im_high)

In [11]:
start_time = timer()
count = mandelbrot_mc_default(i, samples)
end_time = timer()
print(f"Execution time: {np.round(end_time - start_time, 2)} seconds")

Execution time: 0.83 seconds


In [12]:
# # [WARN]: On Linux Mint 20, 16GB RAM, 2GB swap space, the kernel crashes as it runs out of memory for N = 1.0e9
# N = 10**8
# # TODO: save large random arrays
# np.random.seed(10010)
# samples = create_batch_uniform(N, re_low, re_high, im_low, im_high)

In [13]:
# start_time = timer()
# count = mandelbrot_mc_default(i,samples)
# end_time = timer()
# print(end_time - start_time)

In [16]:
s = 10**5
i_values = np.array([10**i for i in range(2,8)], dtype = np.int64)

re_low, re_high = -2., 1.
im_low, im_high = 0, 1.5

npz_filename = "mc_fixed_s.npz"

mean_areas = np.empty(len(i_values))
sample_variances = np.empty(len(i_values))

if not exists(npz_filename):
    number_of_batches = 50
    for k,i in enumerate(i_values):
        areas_batch = np.empty(number_of_batches)
        for j in range(number_of_batches):
            samples = create_batch_uniform(s, re_low, re_high, im_low, im_high)
            iter_counts = mandelbrot_mc_default(i, samples)
            est_area = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
            areas_batch[j] = est_area
        μ, svar = sample_mean_variance(areas_batch)
        mean_areas[k] = μ
        sample_variances[k] = svar
    np.savez(npz_filename, mean_areas, sample_variances)
else:
    save_data = np.load(npz_filename)
    mean_areas = save_data['arr_0']
    sample_variances = save_data['arr_1']
    
confidence_95 = confidence_intervals(sample_variances, p = 0.05)

KeyboardInterrupt: 

In [ ]:
#TODO: plotting

In [ ]:
i = 2500
s_values = np.array([10**s for s in range(2,8)], dtype = np.int64)

re_low, re_high = -2., 1.
im_low, im_high = 0, 1.5

npz_filename = "mc_fixed_i.npz"

mean_areas = np.empty(len(s_values))
sample_variances = np.empty(len(s_values))

if not exists(npz_filename):
    number_of_batches = 50
    for k,s in enumerate(s_values):
        areas_batch = np.empty(number_of_batches)
        for j in range(number_of_batches):
            samples = create_batch_uniform(s, re_low, re_high, im_low, im_high)
            iter_counts = mandelbrot_mc_default(i, samples)
            est_area = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
            areas_batch[j] = est_area
        μ, svar = sample_mean_variance(areas_batch)
        mean_areas[k] = μ
        sample_variances[k] = svar
    np.savez(npz_filename, mean_areas, sample_variances)
else:
    save_data = np.load(npz_filename)
    mean_areas = save_data['arr_0']
    sample_variances = save_data['arr_1']
    
confidence_95 = confidence_intervals(sample_variances, p = 0.05)

In [ ]:
# TODO: plotting

For a fixed $i$, $\lim_{s\to\infty}A_{i,s} = A_i > A_\text{M}$, i.e. we are essentially calculating the area of a superset of the Mandelbrot set, one that includes points that would have met the escape condition after $i$ iterations.

Given a strictly increasing sequence of $i$-values $\{i_1, i_2, \ldots i_n\}$, we would like to pick $i = i_k$ for the smallest $k$ such that $|A_{k+1} - A_{k}|$ is less than some threshold value.

Henceforth, we simulate only the top half of the mandelbrot set.

In [ ]:
i_values = np.array([50, 100, 200, 500, 1000, 2000], dtype = np.int64) #TODO: tweak values
s_values = np.array([10**i for i in range(4,9)], dtype = np.int64)
if not exists("monte_carlo_random_sampling.npz"):
    re_low, re_high = -2., 1.
    im_low, im_high = 0, 1.5

    number_of_batches = 50
    mean_areas = np.zeros((len(i_values), len(s_values)))
    sample_variances = np.zeros((len(i_values), len(s_values)))

    start_time = timer()
    for k,i in enumerate(i_values):
        # repeat `number_of_batches` times for each s-value
        for l,s in enumerate(s_values):
            areas_batch = np.zeros(number_of_batches)
            for j in range(number_of_batches):
                samples = create_batch_uniform(s, re_low, re_high, im_low, im_high)
                iter_counts = mandelbrot_mc_default(i, samples)
                est_area = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
                areas_batch[j] = est_area
            μ, svar = sample_mean_variance(areas_batch)
            mean_areas[k][l] = μ
            sample_variances[k][l] = svar
    end_time = timer()
    np.savez("monte_carlo_random_sampling.npz", mean_areas, sample_variances)
else:
    save_data = np.load("monte_carlo_random_sampling.npz")
    mean_areas = save_data['arr_0']
    sample_variances = save_data['arr_1']

In [ ]:
print((end_time - start_time)/60)

In [ ]:
#n number of batches

def ttest_(means, variances, n):

    critical_value = stats.t.ppf(q=0.975, df=n-1)
    
    t_test = (critical_value * np.sqrt(variances))/ np.sqrt(n)
    
    return t_test, critical_value


In [ ]:
ttest_(means=mean_areas*2, variances=sample_variances*2, n=50)

In [ ]:
def sample_variance_plotter_appendix(simulation_data, title):
    """"
    Plots mean area and standard deviation of the mandelbrot set 
    against sample sizes simulated with different number of 
    iterations
    """
    
    s_values = np.array([10**i for i in range(4,9)], dtype = np.int64)
    
    data = np.load(simulation_data + ".npz")
    
    mean_areas = data['arr_0']*2
    
    sample_variances = data['arr_1']*2
    
    mean_i_50 = mean_areas[0]
    mean_i_100 = mean_areas[1]
    mean_i_200 = mean_areas[2]
    mean_i_500 = mean_areas[3]
    mean_i_1000 = mean_areas[4]
    mean_i_2000 = mean_areas[5]

    std_i_50 = np.sqrt(sample_variances[0])
    std_i_100 = np.sqrt(sample_variances[1])
    std_i_200 = np.sqrt(sample_variances[2])
    std_i_500 = np.sqrt(sample_variances[3])
    std_i_1000 = np.sqrt(sample_variances[4])
    std_i_2000 = np.sqrt(sample_variances[5])


    plt.plot(s_values, mean_i_50, color = "maroon", alpha = 0.6, label = "Approximation with 50 iterations")
    plt.fill_between(s_values, mean_i_50 - std_i_50, mean_i_50 + std_i_50, alpha = 0.3, color = "maroon")

    plt.plot(s_values, mean_i_100, color = "firebrick", alpha = 0.6, label = "Approximation with 100 iterations")
    plt.fill_between(s_values, mean_i_100 - std_i_100, mean_i_100 + std_i_100, alpha = 0.3, color = "firebrick")

    plt.plot(s_values, mean_i_200, color = "tomato", alpha = 0.6, label = "Approximation with 200 iterations")
    plt.fill_between(s_values, mean_i_200 - std_i_200, mean_i_200 + std_i_200, alpha = 0.3, color = "tomato")

    plt.plot(s_values, mean_i_1000, color = "orange", alpha = 0.6, label = "Approximation with 1000 iterations")
    plt.fill_between(s_values, mean_i_1000 - std_i_1000, mean_i_1000 + std_i_1000, alpha = 0.3, color = "orange")

    plt.plot(s_values, mean_i_2000, color = "yellow", alpha = 0.6, label = "Approximation with 2000 iterations")
    plt.fill_between(s_values, mean_i_2000 - std_i_2000, mean_i_2000 + std_i_2000, alpha = 0.3, color = "yellow")

    #plt.hlines(1.50659, s_values[0], s_values[-1], label = "Theoretical Mandelbrot area", linestyles="dashed")

    plt.ylim(top=max(mean_i_50)+0.06) 
    plt.xscale('log')
    plt.xlabel("Samples")
    plt.ylabel("Mandelbrot area")
    plt.grid(True)
    plt.legend(loc="upper right", fontsize="x-small")
    
    plt.savefig(title)
    plt.show() 
    
    
    

In [ ]:
sample_variance_plotter_appendix(simulation_data = "monte_carlo_random_sampling", title = "line_plot_samples_random_appendix.png")

In [ ]:
def iteration_variance_plotter_appendix(simulation_data, title):
    """"
    Plots mean area and standard deviation of the mandelbrot set 
    against sample sizes simulated with different number of 
    iterations
    """
    i_values = np.array([50, 100, 200, 500, 1000, 2000], dtype = np.int64) 
    
    data = np.load(simulation_data + ".npz")
    
    mean_areas = data['arr_0']*2
    
    sample_variances = data['arr_1']*2
    
    mean_s_10_4 = [i[0] for i in mean_areas]
    mean_s_10_5 = [i[1] for i in mean_areas]
    mean_s_10_6 = [i[2] for i in mean_areas]
    mean_s_10_7 = [i[3] for i in mean_areas]
    mean_s_10_8 = [i[4] for i in mean_areas]

    std_s_10_4 = np.sqrt([i[0] for i in sample_variances])
    std_s_10_5 = np.sqrt([i[1] for i in sample_variances])
    std_s_10_6 = np.sqrt([i[2] for i in sample_variances])
    std_s_10_7 = np.sqrt([i[3] for i in sample_variances])
    std_s_10_8 = np.sqrt([i[4] for i in sample_variances])


    plt.plot(i_values, mean_s_10_4, alpha = 0.5, label = "Approximation with $10^4$ samples")
    plt.fill_between(i_values, mean_s_10_4 - std_s_10_4, mean_s_10_4 + std_s_10_4, alpha = 0.2)

    plt.plot(i_values, mean_s_10_5, alpha = 0.5, label = "Approximation with $10^5$ samples")
    plt.fill_between(i_values, mean_s_10_5 - std_s_10_5, mean_s_10_5 + std_s_10_5, alpha = 0.2)
    
    plt.plot(i_values, mean_s_10_6, alpha = 0.5, label = "Approximation with $10^6$ samples")
    plt.fill_between(i_values, mean_s_10_6 - std_s_10_6, mean_s_10_6 + std_s_10_6, alpha = 0.2)
    
    plt.plot(i_values, mean_s_10_7, alpha = 0.5, label = "Approximation with $10^7$ samples")
    plt.fill_between(i_values, mean_s_10_7 - std_s_10_7, mean_s_10_7 + std_s_10_7, alpha = 0.2)
    
    plt.plot(i_values, mean_s_10_8, alpha = 0.5, label = "Approximation with $10^8$ samples")
    plt.fill_between(i_values, mean_s_10_8 - std_s_10_8, mean_s_10_8 + std_s_10_8, alpha = 0.2)
    
    #plt.hlines(1.50659, i_values[0], i_values[-1], label = "Theoretical Mandelbrot area", linestyles="dashed")

    plt.xlabel("Iterations")
    plt.ylabel("Mandelbrot area")
    plt.grid(True)
    plt.legend(loc="upper right", fontsize="x-small")
    
    plt.savefig(title)
    plt.show() 
    

In [ ]:
iteration_variance_plotter_appendix(simulation_data = "monte_carlo_random_sampling", title = "line_plot_iterations_random.png")

In [ ]:
def sample_variance_box_plot(random_sampling_data, lhs_data, orthogonal_sampling_data, title):
    
    box_plot_list = [[], [], []]
    
    if data == np.load(random_sampling_data + ".npz"):
        mean_areas = data['arr_0']*2
        sample_variances = data['arr_1']*2 
        print(mean_i_100 = mean_areas[1])

    

In [ ]:
l_bounds, u_bounds = (0, 0), (1, 1)
nax = 3
nsq = nax * nax
x_randos, y_randos = random_uniform_orthogonal(nax, l_bounds, u_bounds)

In [ ]:
plt.scatter(x_randos, y_randos, s = 0.4, c = "red") #TODO?: colourscheme
plt.hlines(np.arange(nsq)/nsq, 0, 1, alpha=0.4) # TODO?: replace these with plt.grid with controls
plt.vlines(np.arange(nsq)/nsq, 0, 1, alpha=0.4)
plt.hlines(np.arange(nax)/nax, 0, 1)
plt.vlines(np.arange(nax)/nax, 0, 1)
# plt.grid(b = True)
plt.show()

In [ ]:
def sampling_methods(n):
    nx = ny = n
    nsq = n**2
    
    p_r = np.random.random((nsq, 2))
    
    sampler = stats.qmc.LatinHypercube(d=2)
    lhc = sampler.random(nsq)
    
    l_bounds, u_bounds = (0, 0), (1, 1)
    x_randos, y_randos = random_uniform_orthogonal(n, l_bounds, u_bounds)
    
    fig = plt.figure(figsize=(10, 10))
    plt.scatter(p_r[:, 0], p_r[:, 1], color="b", label="Pure random")
    plt.scatter(lhc[:, 0], lhc[:, 1], color="r", label="LHC")
    plt.scatter(x_randos, y_randos, color="g", label="Orthogonal")
    
    plt.xlabel("x")
    plt.ylabel("y")
    plt.hlines(np.arange(nsq)/nsq, 0, 1, alpha = 0.7, color="b")
    plt.vlines(np.arange(nsq)/nsq, 0, 1, alpha = 0.7, color="b")
    plt.hlines(np.arange(n)/n, 0, 1, color="g")
    plt.vlines(np.arange(n)/n, 0, 1, color="g")
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.legend(loc = "best")
    plt.show()

In [ ]:
sampling_methods(n=3)

In [ ]:
i_values = np.array([1000, 2000, 3000, 4000, 5000], dtype = np.int64) #TODO: tweak values
s_values = np.array([10**i for i in range(4,8)], dtype = np.int64)
npz_filename = "mc_random_fullsave.npz"
if not exists(npz_filename):
    re_low, re_high = -2., 1.
    im_low, im_high = 0, 1.5

    number_of_batches = 50
    areas = np.empty((len(i_values),len(s_values), number_of_batches))
    mean_areas = np.zeros((len(i_values), len(s_values)))
    sample_variances = np.zeros((len(i_values), len(s_values)))

    start_time = timer()
    for k,i in enumerate(i_values):
        # repeat `number_of_batches` times for each s-value
        for l,s in enumerate(s_values):
            print(f"Working with i = {i}, s = {s}.")
#             areas_batch = np.zeros(number_of_batches)
            for j in range(number_of_batches):
                samples = create_batch_uniform(s, re_low, re_high, im_low, im_high)
                iter_counts = mandelbrot_mc_default(i, samples)
                est_area = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
                areas[k][l][j] = est_area
            μ, svar = sample_mean_variance(areas[k][l])
            mean_areas[k][l] = μ
            sample_variances[k][l] = svar
    end_time = timer()
    np.savez(npz_filename, mean_areas, sample_variances, areas) 
else:
    save_data = np.load(npz_filename)
    mean_areas = save_data['arr_0']
    sample_variances = save_data['arr_1']
    areas = save_data['arr_2']

In [ ]:
full_random = np.load("mc_random_fullsave.npz")

#print(full_random['arr_0']*2)
print(full_random['arr_2']*2)

In [ ]:
i_values = np.array([1000, 2000, 3000, 4000, 5000], dtype = np.int64) #TODO: tweak values
s_values = np.array([10**i for i in range(4,8)], dtype = np.int64)
def sample_variance_plotter(simulation_data, title):
    """"
    Plots mean area and standard deviation of the mandelbrot set 
    against sample sizes simulated with different number of 
    iterations
    """
    data = np.load(simulation_data + ".npz")
    
    mean_areas = data['arr_0']*2
    
    sample_variances = data['arr_1']*2
    
    mean_i_1000 = mean_areas[0]
    mean_i_2000 = mean_areas[1]
    mean_i_3000 = mean_areas[2]
    mean_i_4000 = mean_areas[3]
    mean_i_5000 = mean_areas[4]


    std_i_1000 = np.sqrt(sample_variances[0])
    std_i_2000 = np.sqrt(sample_variances[1])
    std_i_3000 = np.sqrt(sample_variances[2])
    std_i_4000 = np.sqrt(sample_variances[3])
    std_i_5000 = np.sqrt(sample_variances[4])


    plt.plot(s_values, mean_i_1000, color = "maroon", alpha = 0.6, label = "i = 1000")
    plt.fill_between(s_values, mean_i_1000 - std_i_1000, mean_i_1000 + std_i_1000, alpha = 0.2, color = "maroon")

    plt.plot(s_values, mean_i_2000, color = "firebrick", alpha = 0.6, label = "i = 2000")
    plt.fill_between(s_values, mean_i_2000 - std_i_2000, mean_i_2000 + std_i_2000, alpha = 0.2, color = "firebrick")

    plt.plot(s_values, mean_i_3000, color = "tomato", alpha = 0.6, label = "i = 3000")
    plt.fill_between(s_values, mean_i_3000 - std_i_3000, mean_i_3000 + std_i_3000, alpha = 0.2, color = "tomato")

    plt.plot(s_values, mean_i_4000, color = "orange", alpha = 0.6, label = "i = 4000")
    plt.fill_between(s_values, mean_i_4000 - std_i_4000, mean_i_4000 + std_i_4000, alpha = 0.2, color = "orange")

    plt.plot(s_values, mean_i_5000, color = "yellow", alpha = 0.6, label = "i = 5000")
    plt.fill_between(s_values, mean_i_5000 - std_i_5000, mean_i_5000 + std_i_5000, alpha = 0.2, color = "yellow")

    #plt.hlines(1.50659, s_values[0], s_values[-1], label = "Theoretical Mandelbrot area", linestyles="dashed")

    #plt.ylim(top=max(mean_i_50)+0.06) 
    plt.xscale('log')
    plt.xlabel("Samples")
    plt.ylabel("Mandelbrot area")
    plt.grid(True)
    plt.legend(loc="upper right", fontsize="x-small")
    
    plt.savefig(title)
    plt.show() 

In [ ]:
sample_variance_plotter(simulation_data = "mc_random_fullsave", title = "line_plot_samples_random.png")

In [ ]:
full_random = np.load("mc_random_fullsave.npz")

full_random_variances = full_random['arr_1']*2
full_random_means = full_random['arr_0']*2

print(full_random_means)

confidence_intervals(sample_variances = full_random_variances, p = 0.05)

In [ ]:
full_random_runs_data = full_random['arr_2']*2
# s=10^6
i_1000 = full_random_runs_data[0][1]
i_2000 = full_random_runs_data[1][1]
i_3000 = full_random_runs_data[2][1]
i_4000 = full_random_runs_data[3][1]
i_5000 = full_random_runs_data[4][1]

print(i_5000)

In [ ]:
print("Welch test max iterations and i=1000:")
print(stats.ttest_ind(i_1000, i_5000))
print("Welch test max iterations and i=2000:")
print(stats.ttest_ind(i_2000, i_5000))
print("Welch test max iterations and i=3000:")
print(stats.ttest_ind(i_3000, i_5000))
print("Welch test max iterations and i=4000:")
print(stats.ttest_ind(i_4000, i_5000))

In [ ]:
i_1000_s_10_5 = full_random_runs_data[0][1]
i_1000_s_10_6 = full_random_runs_data[0][2]
i_2000_s_10_5 = full_random_runs_data[1][1]
i_2000_s_10_6 = full_random_runs_data[1][2]
i_3000_s_10_5 = full_random_runs_data[2][1]
i_3000_s_10_6 = full_random_runs_data[2][2]
print(i_1000_s_10_5)

In [ ]:
import scipy

print(scipy.stats.ttest_1samp(i_1000_s_10_5, 1.5065918849))
print(scipy.stats.ttest_1samp(i_1000_s_10_6, 1.5065918849))
print(scipy.stats.ttest_1samp(i_2000_s_10_5, 1.5065918849))
print(scipy.stats.ttest_1samp(i_2000_s_10_6, 1.5065918849))

## Orthogonal Sampling
$s = 3000$ gave us a 95% confidence interval of half-width $ < 10^{-3}$. We expect latin hypercube and orthogonal
sampling methods to give us a significantly smaller confidence interval.

In [ ]:
@njit
def run_batch(s, i, re_low, re_high, im_low, im_high, batch_size, batch_generator):
    areas = np.empty(batch_size, dtype = np.float64)
    for batch in range(batch_size):
        samples = batch_generator(s, re_low, re_high, im_low, im_high)
        iter_counts = mandelbrot_mc_default(i, samples)
        areas[batch] = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
        return areas
    
def run_batch_lhc(s, i, re_low, re_high, im_low, im_high, batch_size, seed = None):
    areas = np.empty(batch_size, dtype = np.float64)
    for batch in range(batch_size):
        samples = create_batch_lhc(s, re_low, re_high, im_low, im_high, seed = seed)
        iter_counts = mandelbrot_mc_default(i, samples)
        areas[batch] = area_from_iter_counts(re_low, re_high, im_low, im_high, iter_counts, i)
        return areas

In [ ]:
s = 10**8
i_values = np.array([3000,4000,5000])

re_low, re_high = -2, 1
im_low, im_high = 0, 1.5

number_of_batches = 50

npz_filename = "mc_orthogonal.npz"
areas = []
if not exists(npz_filename):
    for i in i_values:
        print(f"Working with i = {i}")
        areas.append(run_batch(s, i, re_low, re_high, im_low, im_high, number_of_batches, create_batch_orthogonal))
    stats = [sample_mean_variance(batch) for batch in areas]
    mean_areas = np.array([stat[0] for stat in stats])
    sample_variances = np.array([stat[1] for stat in stats])
    areas = np.array(areas)
    np.savez(npz_filename, mean_areas, sample_variances, areas)
else:
    save_data = np.load(npz_filename)
    keys = [f"arr_{i}" for i in range(3)]
    mean_areas, sample_variances, areas = (save_data[kay] for key in keys)

## Latin Hypercube Sampling

In [ ]:
s = 10**8
i_values = np.array([3000,4000,5000])

re_low, re_high = -2, 1
im_low, im_high = 0, 1.5

number_of_batches = 50

npz_filename = "mc_lhc.npz"
areas = []
if not exists(npz_filename):
    for i in i_values:
        areas.append(run_batch_lhc(s, i, re_low, re_high, im_low, im_high, number_of_batches))
    statistics = [sample_mean_variance(batch) for batch in areas]
    mean_areas = np.array([stat[0] for stat in statistics])
    sample_variances = np.array([stat[1] for stat in statistics])
    areas = np.array(areas)
    np.savez(npz_filename, mean_areas, sample_variances, areas)
else:
    save_data = np.load(npz_filename)
    keys = [f"arr_{i}" for i in range(3)]
    mean_areas, sample_variances, areas = (save_data[kay] for key in keys)

## Comparison between PRS, LHC, and OS
Fix $i, s$ and vary the number of runs

In [ ]:
i = 1000
s = 10**5
batch_sizes = np.array([10**i for i in range(1, 5)])

npz_filename = "sampling_methods_comparison.npz"

if not exists(npz_filename):
    mean_areas = [np.empty(len(batch_sizes)) for _ in range(3)]
    sample_variances = [np.empty(len(batch_sizes)) for _ in range(3)]


    #pure random sampling
    for b in batch_sizes:
        areas = run_batch(s, i, re_low, re_high, im_low, im_high, b, create_batch_uniform)
        μ, σsq = sample_mean_variance(areas)
        mean_areas[0][b], sample_variances[0][b] = μ, σsq

    for b in batch_sizes:
        areas = run_batch(s, i, re_low, re_high, im_low, im_high, b, create_batch_orthogonal)
        μ, σsq = sample_mean_variance(areas)
        mean_areas[1][b], sample_variances[1][b] = μ, σsq

    for b in batch_sizes:
        areas = run_batch_lhc(s, i, re_low, re_high, im_low, im_high, b)
        μ, σsq = sample_mean_variance(areas)
        mean_areas[2][b], sample_variances[2][b] = μ, σsq
    
    mean_areas = np.array(mean_areas)
    sample_variances = np.array(sample_variances)
    
    np.savez(npz_filename, mean_areas, sample_variances)
else:
    save_data = np.load(npz_filename)
    indices = (f"arr_{i}" for i in range(2))
    mean_areas, sample_variances = save_data[i] for i in indices

In [ ]:
create_batch_orthogonal?